# Overview

We have previously loaded up data from The "Hate Speech and Offensive Language" by project Tom Davidson,Dana Warmsley, Michael Macy, and Ingmar Weber. 2017. "Automated Hate Speech Detection and the Problem of Offensive Language." We got the n-grams whose presence Davidson et al found to make it highly probable that the speech in question was hate speech. We then processed out data to count the occurences of any of these particular n-grams in each of our articles. 

Lets no proceed to take a closer look on the distribution of these nrgams in our own dataset. 

# Count of hateful n-grams

First and foremost, I want to tall up the total number of each of the n-grams that were found anywhere in our dataset, regardless of where they were found. This is a simple matter of adding up the columns for each n-gram generated by the CountVectorizer algorithm. We will then generate a bar plot to see which n-grams occur most often. 

In [ ]:
df_hatevecs = df_hatevecs.drop(['hate_score','hate_hits'],axis=1)
df_counts = df_hatevecs.sum(axis=0)
df_counts.sort_values(inplace=True,ascending = False)
df_counts = df_counts[df_counts > 0]
df_counts.plot(kind='barh',figsize=(10,15))

Interesting! This makes sense. We see that the n-grams that occure most often are in fact what we would intuitively expect to see most often in hate speech. I find it particularly interesting that "blame the" is one of the highest occuring n-grams and this makes compelete sense, since peoples need to scapegoat and assign blame is very often at the root of their biases. 

# Distribution of 'hate score'

Now I want to take a closer look at the hate scores. Recall that each article is being assigned a hate score calculated by multiplying the count of each of the hateful n-grams found in it by the probability Davidson and all found of such n-grams indicating hate speech. This way, n-grams that are less likely to indicate hate speech will contribute less towards the articles hate score and vice versa. 

Lets see how the hate scores are distributed across the articles in a histogram.

In [ ]:
df_hatescore = df_hatecounts[df_hatecounts['hate_score'] > 0]
df_hatescore['hate_score'].hist(bins=100)

Looks like most of the articles have a hate score of '1', with the fewer and few articles having higher number of hateful n-grams. This seems about what we would expect. 

Lets take a closer look at articles with a higher hatescore, anything above 2.

In [ ]:
df_hatescore2 = df_hatescore[df_hatescore['hate_score'] > 2 ]
df_hatescore2['hate_score'].hist(bins=100)

# Average hate score per source

Im now curious to see what the average hatescore is for each news source. Are some sources more likely to contain hateful n-grams than others?

In [ ]:
df_source_scores = df_hatecounts[['source','hate_score']].groupby('source').mean()
df_source_scores.sort_values('hate_score',inplace=True)
df_source_scores.plot(kind='barh',figsize=(10,15))

Just eyeballing this it definitely seems to corroborate alot of "conventional wisdom" about hate speech. Generally, hate speech is seen as being a problem associated more with the conservative end of the political spectrum (my apologies if you disagree with this generalization). Sure enough, the two highest hate scores are for "the American conservative" and "the American thinker" , both sources that are solidly towards the right of the political spectrum.
Ofcourse I again want to reiterate , as Terry pointed out, that the presence of terms associated with hate speech do not necessarily indicate that hate speech is being promoted.  An excellent case in point is "Vox", which  has the third highest score. But Vox does a lot of work doing like in depth, embedded, gonzo style journalistic pieces on hate groups like neo Nazis and KKK etc, so you would expect their articles to reference alot of hate speech. But most likely, the hits found on vox will be related to articles discussing or describing hateful speech and conduct and not necessarily articulating it as their own stance. You also see nytimes and nbc news up there, to further highlight the fact that simple occurrence of hateful terminology is not sufficient to arrive at any kind of conclusion.

# Utterance level analysis

In [ ]:
df_utter = pd.read_sql('SELECT * FROM utterances',conn,index_col='index')
df_unlab = pd.read_sql('SELECT * FROM cleaned',conn,index_col='index')
df_article_index = df_unlab.reset_index()[['text','source','link','index']].drop_duplicates(subset='link')
df_article_index.head()

df_utter_redux = df_utter.merge(df_article_index,how='inner',left_on='article_index',right_on='index')

In [ ]:
df_utter_redux['text'][0]

In [ ]:
df_utter_redux.head(70)